In [ ]:
# import os
# import sys
# import numpy as np
# sys.path.append('./src')

# def period_aggregate(obs, period=10, agg="mean"):
#     """
#     obs: (T, 9)
#     返回: (1, 9)
#     """
#     obs = np.asarray(obs, dtype=np.float32).reshape(-1, 9)
#     if agg == "mean":
#         return obs.mean(axis=0, keepdims=True)
#     elif agg == "sum":
#         return obs.sum(axis=0, keepdims=True)
#     elif agg == "last":
#         return obs[-1:].reshape(1, -1)
#     else:
#         raise ValueError(f"Unknown agg={agg}")
    
# def compute_period_stats(episodes, period=10, agg="mean", obs_key="observations"):
#     ps = []
#     for ep in episodes:
#         for obs in ep[obs_key]:
#             P = period_aggregate(obs, period=period, agg=agg)
#             if len(P) > 0:
#                 ps.append(P)
#     X = np.concatenate(ps, axis=0)  # (sum_periods, 9)
#     mean = X.mean(axis=0)
#     var = X.var(axis=0)
#     return mean, var

# episodes_raw = np.load("data/episodes.npz", allow_pickle=True)
# episodes_raw = episodes_raw["episodes"].tolist()
# obs_mean_p, obs_var_p = compute_period_stats(episodes_raw, period=10, agg="mean")
# print("period-mean:", obs_mean_p)
# print("period-var:", obs_var_p)

In [1]:
import os
import sys
import numpy as np
sys.path.append('./src')

obs_mean = np.array([7.9053516e+02, 1.9228922e+04, 3.3525293e+02, 1.1191241e+03, 3.7180111e-01, 1.3477822e+06, 3.9543977e+03, 0.0000000e+00, 9.5033712e+00])
obs_var = np.array([1.53039703e+01, 1.37877227e+04, 1.24639764e-01, 1.58602977e+00, 6.52246754e-08, 2.10554672e+08, 4.33407402e+00, 1.00000000e+00, 6.01958632e-02])
reward_scale = 1e-3

def normalize_obs(obs, period=10, agg="mean"):
    """
    obs: (T, 9)
    返回: (1, 9)
    """
    obs = np.asarray(obs, dtype=np.float32).reshape(-1, 9)
    if agg == "mean":
        obs = (obs - obs_mean) / (np.sqrt(obs_var) + 1e-8)
        return obs.mean(axis=0, keepdims=True)
    else:
        raise ValueError(f"Unknown agg={agg}")

def process_reward(reward):
    return reward * reward_scale

def process_episode(episode):
    return {
        "steps": episode["steps"],
        "rewards": [process_reward(r) for r in episode["rewards"]],
        "health": list(range(len(episode["steps"]) - 1, -1, -1)),
        "observations": [normalize_obs(obs) for obs in episode["observations"]]
    }

episodes = np.load("data/episodes.npz", allow_pickle=True)
episodes = episodes["episodes"].tolist()
episodes = [process_episode(episode) for episode in episodes]

In [2]:
import numpy as np

pairs = []

for episode in episodes:
    obs_seq = episode["observations"]
    health_seq = episode["health"]
    pairs.extend(list(zip(obs_seq, health_seq)))
    
pairs = np.array(pairs, dtype=object)
idx = np.random.permutation(len(pairs))
pairs = pairs[idx]

print(f"Total shuffled pairs: {len(pairs)}")

Total shuffled pairs: 9456


# Linear model

In [3]:
import numpy as np

X = []
y = []

for obs, health in pairs:
    X.append(obs.reshape(-1))      # (1,9) -> (9,)
    y.append(int(health < 3))      # 二分类标签

X = np.stack(X, axis=0)            # (N, 9)
y = np.array(y)                    # (N,)

print("Positive ratio:", y.mean()) # 看一下类别不平衡情况

Positive ratio: 0.24746192893401014


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

clf = LogisticRegression(
    max_iter=2000,
    n_jobs=-1
)

clf.fit(X_train, y_train)

c:\Users\ming\miniconda3\envs\FinRL\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

acc = clf.score(X_test, y_test)
print(f"Test accuracy: {acc:.4f}")

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

y_prob = clf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print(f"ROC-AUC: {auc:.4f}")

coef = clf.coef_[0]
for i, w in enumerate(coef):
    print(f"dim {i}: {w:.3f}")

prob = clf.predict_proba(X_test)[:, 1]
y_pred = (prob >= 0.5)
threshold = 0.3  # 试 0.3 / 0.25 / 0.2

y_pred = (prob >= threshold).astype(int)
print(classification_report(y_test, y_pred, digits=4))

Test accuracy: 0.8980
              precision    recall  f1-score   support

           0     0.9307    0.9340    0.9324      1424
           1     0.7970    0.7885    0.7927       468

    accuracy                         0.8980      1892
   macro avg     0.8638    0.8612    0.8625      1892
weighted avg     0.8976    0.8980    0.8978      1892

ROC-AUC: 0.9558
dim 0: -1.387
dim 1: -1.733
dim 2: 0.356
dim 3: -0.687
dim 4: -0.690
dim 5: -1.190
dim 6: 0.603
dim 7: 0.000
dim 8: 0.643
              precision    recall  f1-score   support

           0     0.9595    0.8827    0.9195      1424
           1     0.7131    0.8868    0.7905       468

    accuracy                         0.8837      1892
   macro avg     0.8363    0.8847    0.8550      1892
weighted avg     0.8986    0.8837    0.8876      1892



# MLP

In [6]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# X: (N, 9), y: (N,)  其中 y = (health < 3).astype(int)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y.numpy()
)

train_ds = TensorDataset(X_tr, y_tr)
test_ds  = TensorDataset(X_te, y_te)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False)

In [7]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)  # logits

In [8]:
# X: (N, 9), y: (N,)  其中 y = (health < 3).astype(int)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y.numpy()
)

train_ds = TensorDataset(X_tr, y_tr)
test_ds  = TensorDataset(X_te, y_te)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False)


device = "cuda" if torch.cuda.is_available() else "cpu"

model = MLP().to(device)

criterion = nn.BCEWithLogitsLoss()  # 二分类
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(150):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        logits = model(xb)
        loss = criterion(logits, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch:02d} | loss = {loss.item():.4f}")

C:\Users\ming\AppData\Local\Temp\ipykernel_38188\3848399420.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
C:\Users\ming\AppData\Local\Temp\ipykernel_38188\3848399420.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


Epoch 00 | loss = 0.3259
Epoch 20 | loss = 0.5462
Epoch 40 | loss = 0.4726
Epoch 60 | loss = 0.1680
Epoch 80 | loss = 0.2384
Epoch 100 | loss = 0.0619
Epoch 120 | loss = 0.0729
Epoch 140 | loss = 0.1555


In [9]:
from sklearn.metrics import roc_auc_score
model.eval()
with torch.no_grad():
    logits = model(X_te.to(device))
    probs = torch.sigmoid(logits).cpu().numpy()

auc = roc_auc_score(y_te.numpy(), probs)
print("MLP ROC-AUC:", auc)

# 还要测试recall, precision, f1-score等指标
from sklearn.metrics import classification_report

for thr in [0.5, 0.4, 0.3, 0.25, 0.2]:
    y_pred = (probs >= thr).astype(int)
    print(f"\nThreshold = {thr}")
    print(classification_report(
        y_te.numpy(), y_pred, digits=4
    ))

MLP ROC-AUC: 0.9563226255641987

Threshold = 0.5
              precision    recall  f1-score   support

         0.0     0.9314    0.9340    0.9327      1424
         1.0     0.7974    0.7906    0.7940       468

    accuracy                         0.8985      1892
   macro avg     0.8644    0.8623    0.8633      1892
weighted avg     0.8982    0.8985    0.8984      1892


Threshold = 0.4
              precision    recall  f1-score   support

         0.0     0.9474    0.9115    0.9291      1424
         1.0     0.7586    0.8462    0.8000       468

    accuracy                         0.8953      1892
   macro avg     0.8530    0.8788    0.8646      1892
weighted avg     0.9007    0.8953    0.8972      1892


Threshold = 0.3
              precision    recall  f1-score   support

         0.0     0.9611    0.8841    0.9210      1424
         1.0     0.7165    0.8910    0.7943       468

    accuracy                         0.8858      1892
   macro avg     0.8388    0.8876    0.8576  

# 

# The challenge

In [20]:
import os
import sys
sys.path.append('./src')

import numpy as np
from dotenv import load_dotenv
from student_client import create_student_gym_env

def get_action(obs, repair_cnt, last_repair_reward):
    obs = normalize_obs(obs).reshape(1, -1)
    prob = clf.predict_proba(obs)[0, 1]

    if prob >= 0.3:
        if last_repair_reward is not None and last_repair_reward < 300:
            return 2  # sell
        return 1  # repair
    return 0  # healthy

load_dotenv()

env = create_student_gym_env()
env.reset()

num_episodes = 20 

episode_total_rewards = []
episode_cum_rewards = []
episode_lengths = []
episode_repair_counts = []

for episode in range(num_episodes):
    print(f"\n--- Episode {episode + 1}/{num_episodes} ---")

    episode_observations = []
    episode_rewards = []
    episode_steps = []

    obs, info = env.reset()
    
    done = False
    
    repair_cnt = 0
    last_repair_reward = None
    check_next_reward = False  # 是否需要检查“修完后的下一步 reward”

    while not done:
        action = get_action(obs, repair_cnt, last_repair_reward)
        print(f"Action taken: {action} (0=healthy, 1=repair, 2=sell)", end=" | ")

        obs, reward, terminated, truncated, info = env.step(action)

        episode_rewards.append(reward)
        print(f"Reward={reward:.2f}, Total Episode Reward={sum(episode_rewards):.2f}")

        done = terminated or truncated

        # ⭐ 如果这是“修完后的下一步”，在这里判断
        if check_next_reward:
            last_repair_reward = reward
            print(f"  -> Last repair reward: {last_repair_reward:.2f}")
            check_next_reward = False

        # ⭐ 如果本步执行的是 repair，只设置标志，不立刻判断
        if action == 1:
            repair_cnt += 1
            print(f"  -> Repair count: {repair_cnt}")
            check_next_reward = True

    total_reward = sum(episode_rewards)
    episode_total_rewards.append(total_reward)
    episode_cum_rewards.append(sum(episode_rewards))
    episode_lengths.append(len(episode_rewards))
    episode_repair_counts.append(repair_cnt)

2026-02-22 12:44:10,573 - student_client.student_gym_env - INFO - Client version 0.3 is newer than latest 0.2
2026-02-22 12:44:10,594 - student_client.student_gym_env - INFO - Created new session: 63cead2d-a08f-49e4-be35-761a1062d165
2026-02-22 12:44:11,481 - student_client.student_gym_env - INFO - Created new episode: 26062db9-6936-43ee-a8d3-7dea65a1b27e
2026-02-22 12:44:11,482 - student_client.student_gym_env - INFO - StudentGymEnv initialized with episode 26062db9-6936-43ee-a8d3-7dea65a1b27e
2026-02-22 12:44:12,503 - student_client.student_gym_env - INFO - Episode 26062db9-6936-43ee-a8d3-7dea65a1b27e reset successfully



--- Episode 1/20 ---


2026-02-22 12:44:13,552 - student_client.student_gym_env - INFO - Episode 26062db9-6936-43ee-a8d3-7dea65a1b27e reset successfully


Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=449.41, Total Episode Reward=449.41
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=401.05, Total Episode Reward=850.46
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=375.15, Total Episode Reward=1225.61
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=327.49, Total Episode Reward=1553.10
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=308.45, Total Episode Reward=1861.56
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=286.17, Total Episode Reward=2147.72
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=224.71, Total Episode Reward=2372.43
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=188.92, Total Episode Reward=2561.35
Action taken: 1 (0=healthy, 1=repair, 2=sell) | Reward=-515.52, Total Episode Reward=2045.82
  -> Repair count: 1
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=421.95, Total Episode Reward=2467.78
  -> Last repair reward: 421.95
Action taken: 0 (0=healthy, 

2026-02-22 12:45:19,103 - student_client.student_gym_env - INFO - Episode 26062db9-6936-43ee-a8d3-7dea65a1b27e reset successfully


...

2026-02-22 12:49:18,813 - student_client.student_gym_env - INFO - Episode 26062db9-6936-43ee-a8d3-7dea65a1b27e reset successfully


Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=441.16, Total Episode Reward=441.16
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=416.73, Total Episode Reward=857.89
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=371.19, Total Episode Reward=1229.08
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=301.81, Total Episode Reward=1530.89
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=304.87, Total Episode Reward=1835.76
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=244.03, Total Episode Reward=2079.79
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=207.39, Total Episode Reward=2287.18
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Episode 26062db9-6936-43ee-a8d3-7dea65a1b27e reached termination state, reason: failure
Reward=-1000.00, Total Episode Reward=1287.18

--- Episode 8/20 ---


2026-02-22 12:49:31,111 - student_client.student_gym_env - INFO - Episode 26062db9-6936-43ee-a8d3-7dea65a1b27e reset successfully


Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=457.70, Total Episode Reward=457.70
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=436.34, Total Episode Reward=894.03
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=365.88, Total Episode Reward=1259.91
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=324.04, Total Episode Reward=1583.95
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=285.61, Total Episode Reward=1869.55
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=250.98, Total Episode Reward=2120.53
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=244.26, Total Episode Reward=2364.79
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=175.17, Total Episode Reward=2539.96
Action taken: 1 (0=healthy, 1=repair, 2=sell) | Reward=-525.81, Total Episode Reward=2014.15
  -> Repair count: 1
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=434.31, Total Episode Reward=2448.46
  -> Last repair reward: 434.31
Action taken: 0 (0=healthy, 

KeyboardInterrupt: 